# Importing all the important libraries

In [17]:
import numpy as np
import Nodes
import datetime
import collections

# Importing the incomplete grid into the notebook

In [18]:
Data = []
with open("Sudoku_sol_1.txt","r") as file:
    for x in range(0,9): 
        Data.append(file.readline().split())
Data = np.array(Data,dtype=np.int32)
print(Data)

[[4 8 3 9 2 1 6 5 7]
 [9 6 7 3 4 5 8 2 1]
 [0 5 1 8 0 6 4 9 3]
 [5 4 8 1 3 2 9 0 6]
 [7 2 9 5 6 4 1 3 8]
 [1 3 6 0 9 8 2 4 5]
 [3 7 2 6 8 9 5 1 4]
 [8 1 4 2 5 3 7 6 9]
 [6 9 5 4 1 7 3 8 0]]


# The sub square violation method which checks if any violations are present within each 9 x 9 sub square

In [19]:
def sub_square(arr):
    data = arr.reshape(3,3,3,3)
    for x in data:
        test = []
        for y in x:
            try:
                test[0] = np.concatenate((test[0], y[0]), axis=0)
                test[1] = np.concatenate((test[1], y[1]), axis=0)
                test[2] = np.concatenate((test[2], y[2]), axis=0)
            except:
                test.append(np.array(y[0]))
                test.append(np.array(y[1]))
                test.append(np.array(y[2]))

        for i in test:
            flag = [item for item, count in collections.Counter(i).items() if count > 1]
            for i in flag:
                if(i != 0): return True
    return False

# The column violation method which checks if there are duplicate numbers in any column within the grid

In [20]:
def is_col_violation(Data):
    count = 0
    while count < 9:
        col = []
        for x in Data:
            if(x[count] == 0): continue
            if(x[count] in col):return True
            col.append(x[count])
        count+=1
    return False

# The row violation method checks if there are any duplicate numbers in each row of the grid

In [21]:
def is_row_violation(Data):
    for x in Data:
        flag = [item for item, count in collections.Counter(list(x)).items() if count > 1]
        for i in flag:
            if(i != 0): return True
    return False

# The get zeros and has zeros methods below provide the positions of all the incomplete cells within the grid and the has zeros method checks if the grid is incomplete (no zeros) or complete (zeros present)

In [22]:
def get_zeros(state):
        zero = np.where(state == 0)
        return zero

def has_zeros(state):

        flag = np.where(state == 0)
        if(len(flag[0]) != 0): return True
        return False

# The depth first search algorithm

In [23]:
def Depth_First_Search_infinite_time (states,Root,limit):
        if(has_zeros(Root.state) == False and is_col_violation(Root.state) == False and is_row_violation(Root.state) == False and sub_square(Root.state) == False):
            print(states)
            return Root.state

        elif(limit == 0):
            return 0

        else:
            cuttoff = False
            zeros = get_zeros(Root.state) 

            for i in range (0,len(Root.children)):
                action = Root.get_child()
                for y,x in enumerate(zeros[0]):
                    child = Nodes.Node(Root.state.copy())
                    child.set_state(zeros[0][y],zeros[1][y], action)
                    states += 1
                    result = Depth_First_Search_infinite_time(states+1,child,limit-1)
                    try:
                            if(result.any()): 
                                return result
                    except:
                            if(result) == 0:
                                cuttoff = True
        if(cuttoff):
            return cuttoff
        
Root_Node = Nodes.Node(Data)
start = datetime.datetime.now()
print(Depth_First_Search_infinite_time(1,Root_Node,len(get_zeros(Root_Node.state)[0])))
end = datetime.datetime.now()
print("\nThe Exhaustive search time is :{}".format((end - start).microseconds/1000))

59
[[4 8 3 9 2 1 6 5 7]
 [9 6 7 3 4 5 8 2 1]
 [2 5 1 8 7 6 4 9 3]
 [5 4 8 1 3 2 9 7 6]
 [7 2 9 5 6 4 1 3 8]
 [1 3 6 7 9 8 2 4 5]
 [3 7 2 6 8 9 5 1 4]
 [8 1 4 2 5 3 7 6 9]
 [6 9 5 4 1 7 3 8 2]]

The Exhaustive search time is :91.73


# The backtracking CSP algorithm

In [24]:
def Backtracking_CSP(States,Root):

        if(has_zeros(Root.state) == False):
            print(States)
            return Root.state

        else:
            zeros = get_zeros(Root.state) 

            for i in range (0,len(Root.children)):
                action = Root.get_child()
                child = Nodes.Node(Root.state.copy())
                child.set_state(zeros[0][0],zeros[1][0], action)

                if(is_col_violation(child.state) == False and is_row_violation(child.state) == False and sub_square(child.state) == False):
                        result = Backtracking_CSP(States+1,child)
                        try:
                            if(result.any()):
                                return result
                        except:
                            pass

Root_Node = Nodes.Node(Data)
start = datetime.datetime.now()
data = Backtracking_CSP(1,Root_Node)
print(data)
end = datetime.datetime.now()
print("\nThe Backtracking_CSP time is :{}".format((end - start).microseconds/1000))

6
[[4 8 3 9 2 1 6 5 7]
 [9 6 7 3 4 5 8 2 1]
 [2 5 1 8 7 6 4 9 3]
 [5 4 8 1 3 2 9 7 6]
 [7 2 9 5 6 4 1 3 8]
 [1 3 6 7 9 8 2 4 5]
 [3 7 2 6 8 9 5 1 4]
 [8 1 4 2 5 3 7 6 9]
 [6 9 5 4 1 7 3 8 2]]

The Backtracking_CSP time is :9.976
